In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from nltk.corpus import stopwords
import nltk
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
# 1. Загрузка данных
df = pd.read_csv('Go Russia.csv')

# 2. Предварительная проверка данных
print("Первые 5 строк:")
print(df.head())
print("\nКолонки датафрейма:")
print(df.columns)
print("\nИнформация о датафрейме:")
df.info()

# 3. Обработка пропущенных значений
df = df.dropna(subset=['Текст'])
df['Текст'] = df['Текст'].astype(str)

# 4. Рассчёт основных параметров
df['char_length'] = df['Текст'].apply(len)  # Длина в символах
df['word_count'] = df['Текст'].apply(lambda x: len(x.split()))  # Количество слов
df['unique_words'] = df['Текст'].apply(lambda x: len(set(x.split())))  # Уникальные слова
df['lexical_diversity'] = df.apply(
    lambda x: x['unique_words'] / x['word_count'] if x['word_count'] > 0 else 0,
    axis=1
)  # Лексическое разнообразие
df['avg_word_length'] = df['Текст'].apply(
    lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0
)  # Средняя длина слова

# 5. Статистика по датафрейму
total_texts = len(df)
avg_char_length = df['char_length'].mean()
avg_word_count = df['word_count'].mean()

print("\n" + "="*50)
print(f"Общее количество текстов: {total_texts}")
print(f"Средняя длина текста в символах: {avg_char_length:.2f}")
print(f"Среднее количество слов в тексте: {avg_word_count:.2f}")
print(f"Среднее количество уникальных слов: {df['unique_words'].mean():.2f}")
print(f"Среднее лексическое разнообразие: {df['lexical_diversity'].mean():.4f}")
print(f"Средняя длина слова: {df['avg_word_length'].mean():.2f} символов")
print("="*50)

In [ ]:
!pip install -U spacy
!python -m spacy download ru_core_news_sm


nltk.download('stopwords')
nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])
russian_stopwords = stopwords.words('russian')

In [ ]:
#  Функция предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление спецсимволов и цифр
    text = re.sub(r'[^а-яё\s]', '', text, flags=re.IGNORECASE)

    # Токенизация и лемматизация с помощью spacy
    doc = nlp(text)
    lemmas = [
        token.lemma_ for token in doc
        if token.text not in russian_stopwords
        and len(token.text) > 2
        and token.is_alpha
    ]

    return ' '.join(lemmas)

#  Предобработка текста
print("\nНачало предобработки текста...")
combined_texts = (df['Заголовок'].fillna('') + ' ' + df['Текст'].fillna(''))
tqdm.pandas(desc="Предобработка текстов")
df['processed_text'] = combined_texts.progress_apply(preprocess_text)
print("Предобработка текста завершена!")

#  Создание эмбеддингов
print("\nСоздание векторных представлений текстов...")
texts = df['processed_text'].tolist()
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
print(f"Создано {len(embeddings)} векторных представлений размерностью {embeddings.shape[1]}")

In [ ]:
# сравнение текстов
print("\nПример текста до обработки:")
print(combined_texts.iloc[0])
print("\nПример текста после обработки:")
print(df['processed_text'].iloc[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
# ---- ФУНКЦИЯ СИЛУЭТНОГО ГРАФИКА ----
def plot_silhouettes(samples_silhouettes, labels, n_clusters, silhouette_avg):
    fig, ax = plt.subplots(figsize=(6, 6))
    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = samples_silhouettes[labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = plt.cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                         0, ith_cluster_silhouette_values,
                         facecolor=color, edgecolor=color, alpha=0.7)

        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # 10 for spacing between silhouette plots

    ax.set_title("Silhouette plot")
    ax.set_xlabel("Silhouette score")
    ax.set_ylabel("Cluster label")
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax.set_yticks([])
    ax.set_xticks(np.arange(-0.2, 1.1, 0.2))
    plt.show()

# ---- ПОИСК ОПТИМАЛЬНОГО ЧИСЛА КЛАСТЕРОВ, СИЛУЭТНЫЕ ГРАФИКИ ----
K = range(2, 8)
inertias = []
silhouette_avgs = []

for k in K:
    print(f"\nОбработка для {k} кластеров...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings)
    inertia = kmeans.inertia_
    silhouette_avg = silhouette_score(embeddings, labels)
    silhouette_avgs.append(silhouette_avg)
    inertias.append(inertia)
    print(f"Кластеров: {k} | inertia: {inertia:.0f} | Silhouette avg: {silhouette_avg:.3f}")

    # Считаем силуэты для всех точек
    samples_silhouettes = silhouette_samples(embeddings, labels)

    # Визуализация силуэтного графика для данного k
    plot_silhouettes(samples_silhouettes, labels, k, silhouette_avg)

# Графики локтя и силуэта
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(K, inertias, marker='o')
plt.xlabel('Число кластеров (k)')
plt.ylabel('Inertia')
plt.title('Метод локтя')

plt.subplot(1, 2, 2)
plt.plot(K, silhouette_avgs, marker='o')
plt.xlabel('Число кластеров (k)')
plt.ylabel('Средний силуэт')
plt.title('Silhouette Score')
plt.tight_layout()
plt.show()

# ВЫБОР ЧИСЛА КЛАСТЕРОВ по анализу графиков
NUM_CLUSTERS = int(input("\nВведите оптимальное число кластеров на основе графиков: "))

# 4. Кластеризация с выбранным числом кластеров
print(f"\nКластеризация с {NUM_CLUSTERS} кластерами...")
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
clusters = kmeans.fit_predict(embeddings)
df['cluster'] = clusters

# 5. Визуализация на плоскости
print("Снижение размерности с помощью t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=40)
emb_2d = tsne.fit_transform(embeddings)

df['tsne_1'] = emb_2d[:, 0]
df['tsne_2'] = emb_2d[:, 1]

plt.figure(figsize=(10,8))
sns.scatterplot(x=df['tsne_1'], y=df['tsne_2'], hue=clusters, palette='tab10', legend='full')
plt.title('Кластеризация текстов о путешествиях')
plt.show()

# 6. Сохраняем таблицу
df.to_csv('Go_Russia_clusters.csv', index=False)
print("Результаты сохранены в Go_Russia_clusters.csv")

# 7. Просмотр примеров из каждого кластера
print("\nПримеры текстов по кластерам:")
for i in range(NUM_CLUSTERS):
    cluster_size = len(df[df['cluster'] == i])
    print(f"\nCluster {i} ({cluster_size} текстов):")
    cluster_examples = df[df['cluster'] == i]['Заголовок'].head(5).tolist()
    for j, example in enumerate(cluster_examples, 1):
        print(f' {j}. {example}')

In [ ]:
!pip install stop-words
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

In [ ]:
# функция для анализа кластеров
def show_cluster_info(df, text_col='processed_text', title_col='Заголовок', n_top_words=15):
    russian_stop_words = get_stop_words('russian')

    print("\n" + "="*50)
    print("АНАЛИЗ КЛАСТЕРОВ")
    print("="*50)

    cluster_stats = []

    for c in sorted(df['cluster'].unique()):
        cluster_df = df[df['cluster'] == c]
        cluster_size = len(cluster_df)
        cluster_stats.append((c, cluster_size))

        print(f"\n### Кластер {c} ({cluster_size} текстов, {cluster_size/total_texts:.1%} данных):")

        # Извлекаем тексты кластера
        cluster_texts = cluster_df[text_col].dropna().astype(str).tolist()

        if not cluster_texts:
            print('Нет текстов для анализа!')
            continue

        # Анализ ключевых слов через TF-IDF
        try:
            vectorizer = TfidfVectorizer(max_features=500, stop_words=russian_stop_words)
            tfidf_matrix = vectorizer.fit_transform(cluster_texts)
            words = vectorizer.get_feature_names_out()
            scores = tfidf_matrix.mean(axis=0).A1
            indices = scores.argsort()[::-1][:n_top_words]
            top_words = [words[i] for i in indices]

            print(f'🔑 Ключевые слова: {", ".join(top_words)}')
        except ValueError:
            print("⚠️ Не удалось извлечь ключевые слова (возможно, недостаточно разнообразная лексика)")

        # Примеры заголовков
        titles = cluster_df[title_col].dropna().astype(str).tolist()[:20]
        print("\n📝 Примеры заголовков:")
        for i, title in enumerate(titles, 1):
            print(f" {i}. {title}")

    # Визуализация распределения кластеров
    plt.figure(figsize=(10, 6))
    clusters, counts = zip(*sorted(cluster_stats, key=lambda x: x[0]))
    plt.bar([str(c) for c in clusters], counts, color=plt.cm.tab10.colors[:len(clusters)])
    plt.title('Распределение текстов по кластерам')
    plt.xlabel('Номер кластера')
    plt.ylabel('Количество текстов')
    plt.grid(axis='y', alpha=0.3)
    plt.show()

# Вызываем функцию анализа кластеров
show_cluster_info(df)

# Дополнительный анализ: средние характеристики по кластерам
print("\n" + "="*50)
print("СРЕДНИЕ ХАРАКТЕРИСТИКИ ПО КЛАСТЕРАМ")
print("="*50)

cluster_metrics = df.groupby('cluster').agg({
    'char_length': 'mean',
    'word_count': 'mean',
    'lexical_diversity': 'mean',
    'avg_word_length': 'mean'
}).reset_index()

cluster_metrics.columns = ['Кластер', 'Символы', 'Слова', 'Лекс.разнообр.', 'Ср.длина слова']
print(cluster_metrics.round(2))

# Визуализация метрик
plt.figure(figsize=(12, 8))
for i, metric in enumerate(['Символы', 'Слова', 'Лекс.разнообр.', 'Ср.длина слова'], 1):
    plt.subplot(2, 2, i)
    sns.barplot(x='Кластер', y=metric, data=cluster_metrics, palette='tab10')
    plt.title(f'Среднее значение: {metric}')
    plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ВЫБОР ЧИСЛА КЛАСТЕРОВ по анализу графиков
NUM_CLUSTERS = int(input("\nВведите оптимальное число кластеров на основе графиков: "))

# 4. Кластеризация с выбранным числом кластеров
print(f"\nКластеризация с {NUM_CLUSTERS} кластерами...")
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
clusters = kmeans.fit_predict(embeddings)
df['cluster'] = clusters

# 5. Визуализация на плоскости
print("Снижение размерности с помощью t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=40)
emb_2d = tsne.fit_transform(embeddings)

df['tsne_1'] = emb_2d[:, 0]
df['tsne_2'] = emb_2d[:, 1]

plt.figure(figsize=(10,8))
sns.scatterplot(x=df['tsne_1'], y=df['tsne_2'], hue=clusters, palette='tab10', legend='full')
plt.title('Кластеризация текстов о путешествиях')
plt.show()

# 6. Сохраняем таблицу
df.to_csv('Go_Russia_clusters.csv', index=False)
print("Результаты сохранены в Go_Russia_clusters.csv")

# 7. Просмотр примеров из каждого кластера
print("\nПримеры текстов по кластерам:")
for i in range(NUM_CLUSTERS):
    cluster_size = len(df[df['cluster'] == i])
    print(f"\nCluster {i} ({cluster_size} текстов):")
    cluster_examples = df[df['cluster'] == i]['Заголовок'].head(5).tolist()
    for j, example in enumerate(cluster_examples, 1):
        print(f' {j}. {example}')

In [ ]:
# функция для анализа кластеров
def show_cluster_info(df, text_col='processed_text', title_col='Заголовок', n_top_words=15):
    russian_stop_words = get_stop_words('russian')

    print("\n" + "="*50)
    print("АНАЛИЗ КЛАСТЕРОВ")
    print("="*50)

    cluster_stats = []

    for c in sorted(df['cluster'].unique()):
        cluster_df = df[df['cluster'] == c]
        cluster_size = len(cluster_df)
        cluster_stats.append((c, cluster_size))

        print(f"\n### Кластер {c} ({cluster_size} текстов, {cluster_size/total_texts:.1%} данных):")

        # Извлекаем тексты кластера
        cluster_texts = cluster_df[text_col].dropna().astype(str).tolist()

        if not cluster_texts:
            print('Нет текстов для анализа!')
            continue

        # Анализ ключевых слов через TF-IDF
        try:
            vectorizer = TfidfVectorizer(max_features=500, stop_words=russian_stop_words)
            tfidf_matrix = vectorizer.fit_transform(cluster_texts)
            words = vectorizer.get_feature_names_out()
            scores = tfidf_matrix.mean(axis=0).A1
            indices = scores.argsort()[::-1][:n_top_words]
            top_words = [words[i] for i in indices]

            print(f'🔑 Ключевые слова: {", ".join(top_words)}')
        except ValueError:
            print("⚠️ Не удалось извлечь ключевые слова (возможно, недостаточно разнообразная лексика)")

        # Примеры заголовков
        titles = cluster_df[title_col].dropna().astype(str).tolist()[:20]
        print("\n📝 Примеры заголовков:")
        for i, title in enumerate(titles, 1):
            print(f" {i}. {title}")

    # Визуализация распределения кластеров
    plt.figure(figsize=(10, 6))
    clusters, counts = zip(*sorted(cluster_stats, key=lambda x: x[0]))
    plt.bar([str(c) for c in clusters], counts, color=plt.cm.tab10.colors[:len(clusters)])
    plt.title('Распределение текстов по кластерам')
    plt.xlabel('Номер кластера')
    plt.ylabel('Количество текстов')
    plt.grid(axis='y', alpha=0.3)
    plt.show()

# Вызываем функцию анализа кластеров
show_cluster_info(df)

# Дополнительный анализ: средние характеристики по кластерам
print("\n" + "="*50)
print("СРЕДНИЕ ХАРАКТЕРИСТИКИ ПО КЛАСТЕРАМ")
print("="*50)

cluster_metrics = df.groupby('cluster').agg({
    'char_length': 'mean',
    'word_count': 'mean',
    'lexical_diversity': 'mean',
    'avg_word_length': 'mean'
}).reset_index()

cluster_metrics.columns = ['Кластер', 'Символы', 'Слова', 'Лекс.разнообр.', 'Ср.длина слова']
print(cluster_metrics.round(2))

# Визуализация метрик
plt.figure(figsize=(12, 8))
for i, metric in enumerate(['Символы', 'Слова', 'Лекс.разнообр.', 'Ср.длина слова'], 1):
    plt.subplot(2, 2, i)
    sns.barplot(x='Кластер', y=metric, data=cluster_metrics, palette='tab10')
    plt.title(f'Среднее значение: {metric}')
    plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()